# Compressive video via Pulsed Illumination

## Setup

*   Preparing repository
*   Download DAVIS2017 dataset
*   Install extra dependencies

In [ ]:
!git clone https://github.com/FOGuzman/PulseIlluminationVideo.git
%cd PulseIlluminationVideo/code
!wget https://data.vision.ee.ethz.ch/csergi/share/davis/DAVIS-2017-trainval-480p.zip
!mkdir -p ./dataset/
!unzip DAVIS-2017-trainval-480p.zip -d ./dataset/
!wget https://github.com/dajes/frame-interpolation-pytorch/releases/download/v1.0.0/film_net_fp32.pt
!mv film_net_fp32.pt ./models/
!pwd
!pip install einops==0.6.1
!pip install addict==2.4.0
!pip install timm==0.9.2

Import modules

In [5]:
import os
import os.path as osp
import sys 
import importlib
#BASE_DIR = osp.dirname(osp.dirname(osp.abspath(__file__)))
#sys.path.append(BASE_DIR)
from torch.utils.data import DataLoader
from config.dataset.builder import build_dataset
from config.utils.mask import generate_masks
from config.utils.logger import Logger
from config.utils.utils import save_image, load_checkpoints, get_device_info
from config.utils.eval_full import eval_psnr_ssim
from config.util_functions import *
import torch
from torch.utils.tensorboard import SummaryWriter
import time
import argparse 
import json 

[Errno 2] No such file or directory: 'PulseIlluminationVideo/code'
/content/PulseIlluminationVideo/code


Set-up parameters

In [6]:
parser = argparse.ArgumentParser() 
args = parser.parse_args(args=[])
args.config                           =  './config/default_config.py'
args.work_dir                         =  '/base_model/'
args.train_dataset_path               =  './dataset/DAVIS/JPEGImages/480p/'
args.test_dataset_path                =  './dataset/test_dataset/'
args.mask_path                        =  './masks/shutter_mask16.mat'
args.model_module                     =  'base_model'
args.loss_module                      =  'loss_test2'
args.gpu                              =  "0"
args.resolution                       =  [128,128]
args.frames                           =  16
args.dataset_crop                     =  [64,64]
args.resume                           =  None
args.Epochs                           =  400
args.batch_size                       =  2
args.learning_rate                    =  0.0001
args.saveImageEach                    =  500
args.saveModelEach                    =  1
args.checkpoints                      =  None
args.device                           =  "cuda"


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=args.gpu
n_gpu = torch.cuda.device_count()
print(torch.cuda.is_available())
print('The number of GPU is {} using {}'.format(n_gpu,args.gpu))

True
The number of GPU is 1 using 0


Start training

In [ ]:
if __name__ == '__main__':

    cfg = update_cfg(args)
    
    logger = Logger(cfg.log_dir)
    writer = SummaryWriter(log_dir = cfg.show_dir)

    dash_line = '-' * 80 + '\n'
    device_info = get_device_info()
    env_info = '\n'.join(['{}: {}'.format(k,v) for k, v in device_info.items()])

    ## Load cuda devide
    device = args.device

    ## Import model
    CNNmethod = importlib.import_module('models.'+ args.model_module)
    model = CNNmethod.cnnModel(frames=args.frames).to(args.device)

    ## Import Loss()
    LossMethod = importlib.import_module('losses.'+ args.loss_module)
    loss_function = LossMethod.Loss(cfg).to(args.device)
    
    ## Optimizer
    optimizer = torch.optim.AdamW([{'params': model.parameters()}], lr=args.learning_rate)

    logger.info('GPU info:\n' 
                + dash_line + 
                env_info + '\n' +
                dash_line)
    logger.info('cfg info:\n'
                + dash_line + 
                json.dumps(cfg, indent=4)+'\n'+
                dash_line) 
    logger.info('Model info:\n'
                + dash_line + 
                str(model)+'\n'+
                dash_line)


    ## Preparing dataset
    mask,mask_s = generate_masks(cfg.train_data.mask_path,cfg.train_data.mask_shape)
    train_data = build_dataset(cfg.train_data,{"mask":mask})
    test_data = build_dataset(cfg.test_data,{"mask":mask})
    train_data_loader = DataLoader(dataset=train_data, 
                                        batch_size=cfg.data.samples_per_gpu,
                                        shuffle=True,
                                        num_workers = cfg.data.workers_per_gpu)
    
    
    start_epoch = 0
    if cfg.checkpoints is not None:
            logger.info("Load pre_train model...")
            resume_dict = torch.load(cfg.checkpoints)
            if "model_state_dict" not in resume_dict.keys():
                model_state_dict = resume_dict
            else:
                model_state_dict = resume_dict["model_state_dict"]
            load_checkpoints(model,model_state_dict)
    else:            
            logger.info("No pre_train model")

    if cfg.resume is not None:
            logger.info("Load resume...")
            resume_dict = torch.load(cfg.resume)
            start_epoch = resume_dict["epoch"]
            model_state_dict = resume_dict["model_state_dict"]
            load_checkpoints(model,model_state_dict)

            optim_state_dict = resume_dict["optim_state_dict"]
            optimizer.load_state_dict(optim_state_dict)


    iter_num = len(train_data_loader) 
    for epoch in range(start_epoch,cfg.runner.max_epochs):
        epoch_loss = 0
        model = model.train()
        start_time = time.time()
        for iteration, data in enumerate(train_data_loader):
            gt, meas = data
            gt = gt.float().to(args.device)
            meas = meas.unsqueeze(1).float().to(args.device)
            batch_size = meas.shape[0]

            optimizer.zero_grad()
            meas_f = torch.cat((gt[:,0:1,:,:],meas,gt[:,-1:,:,:]),1)

            model_out = model(meas_f,args)
            model_out_f = torch.cat((gt[:,0:1,:,:],model_out[:,1:-1,:,:],gt[:,-1:,:,:]),1)
            

            if not isinstance(model_out,list):
                model_out = [model_out_f]
            loss = loss_function(model_out_f, gt)
            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()


            if (iteration % cfg.log_config.interval) == 0:
                lr = optimizer.state_dict()["param_groups"][0]["lr"]
                iter_len = len(str(iter_num))
                logger.info("epoch: [{}][{:>{}}/{}], lr: {:.6f}, loss: {:.5f}.".format(epoch,iteration,iter_len,iter_num,lr,loss.item()))
                writer.add_scalar("loss",loss.item(),epoch*len(train_data_loader) + iteration)
                
            if (iteration % cfg.save_image_config.interval) == 0:
                sing_out = model_out_f[0].detach().cpu().numpy()
                sing_gt = gt[0].cpu().numpy()
                sing_mask = mask
                image_name = osp.join(cfg.train_image_save_dir,str(epoch)+"_"+str(iteration)+".png")
                save_image(sing_out,sing_gt,sing_mask,image_name)
        end_time = time.time()

        logger.info("epoch: {}, avg_loss: {:.5f}, time: {:.2f}s.\n".format(epoch,epoch_loss/(iteration+1),end_time-start_time))

        if (epoch % cfg.checkpoint_config.interval) == 0:
 
            save_model = model
            checkpoint_dict = {
                "epoch": epoch, 
                "model_state_dict": save_model.state_dict(), 
                "optim_state_dict": optimizer.state_dict(), 
            }
            torch.save(checkpoint_dict,osp.join(cfg.checkpoints_dir,"epoch_"+str(epoch)+".pth")) 

        if epoch % cfg.eval.interval==0:
            psnr_dict,ssim_dict = eval_psnr_ssim(model,test_data,mask,mask_s,args)
            psnr_str = ", ".join([key+": "+"{:.4f}".format(psnr_dict[key]) for key in psnr_dict.keys()])           
            ssim_str = ", ".join([key+": "+"{:.4f}".format(ssim_dict[key]) for key in ssim_dict.keys()])
            psnr_values = list(psnr_dict.values())
            ssim_values = list(ssim_dict.values())
            logger.info("Mean PSNR: \n{}.\n".format(psnr_str))
            logger.info("Mean SSIM: \n{}.\n".format(ssim_str))
            writer.add_scalar("psnr_metric",psnr_values[-1],epoch*len(train_data_loader) + iteration)
            writer.add_scalar("ssim_metric",ssim_values[-1],epoch*len(train_data_loader) + iteration)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
INFO:root:GPU info:
--------------------------------------------------------------------------------
CUDA available: True
GPU numbers: 1
GPU INFO: [{'GPU 0': 'Tesla T4'}]
--------------------------------------------------------------------------------

2023-06-09 08:42:02,844 - <ipython-input-7-374883abfb46> [line: 26] - GPU info:
--------------------------------------------------------------------------------
CUDA available: True
GPU numbers: 1
GPU INFO: [{'GPU 0': 'Tesla T4'}]
--------------------------------------------------------------------------------

INFO:root:cfg info:
--------------------------------------------------------------------------------
{
    "test_data": {
